In [10]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
!rm -rf ./logs/

In [12]:
# Import TensorFlow and the TensorBoard HParams plugin;
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [13]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Experiment setup and the HParams experiment summary

Experiment with three hyperparameters in the model:
1. Number of units in the first layer
2. Dropout rate in the dropout layer
3. Optimizer

List the values to try, and log an experiment configuration to TensorBoard. This step is optional: you can provide domain information to enable more precise filtering of hyperparameters in the UI, and you can specify which metrics should be displayed.

In [19]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [20]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ])
    
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=10)
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [21]:
# For each run, log an hparams summary with the hyperparameters and final accuracy.
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams) # Record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

### Start runs and log them all one parent directory

You can now try multiple experiments, training each one with a different set of hypterparameters.

For simplicity, use a grid search: try all combination of the discrete parameters and just the lower and upper bounds of the real-valued parameter. For more complex scenarios, it might be more efficient to choose each hypterparameter value randomly (this is called random search). There are more advanced methods can be used.

Run a few experiments, which will take a few minutes:

In [22]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = 'run-%d' % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.7068 - accuracy: 0.7588
Epoch 2/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.5087 - accuracy: 0.8200
Epoch 3/10
60000/60000 [==============================] - 3s 42us/sample - loss: 0.4734 - accuracy: 0.8318
Epoch 4/10
60000/60000 [==============================] - 2s 41us/sample - loss: 0.4496 - accuracy: 0.8394
Epoch 5/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4374 - accuracy: 0.8425
Epoch 6/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.4280 - accuracy: 0.8468
Epoch 7/10
60000/60000 [==============================] - 2s 42us/sample - loss: 0.4214 - accuracy: 0.8484
Epoch 8/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4151 - accuracy: 0.8491s - los
Epoch 9/10
60000/60000 [=========

--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.9885 - accuracy: 0.6553
Epoch 2/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.6716 - accuracy: 0.7638
Epoch 3/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.6042 - accuracy: 0.7882
Epoch 4/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5611 - accuracy: 0.8017
Epoch 5/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5367 - accuracy: 0.8102
Epoch 6/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5200 - accuracy: 0.8156
Epoch 7/10
60000/60000 [==============================] - 2s 32us/sample - loss: 0.5073 - accuracy: 0.8196
Epoch 8/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.4972 - accuracy: 0.8224
Epoch 9/10
60000/60000 [=================

--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.7904 - accuracy: 0.7202
Epoch 2/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.5878 - accuracy: 0.7863
Epoch 3/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.5456 - accuracy: 0.7998
Epoch 4/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.5276 - accuracy: 0.8083
Epoch 5/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.5124 - accuracy: 0.8137
Epoch 6/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.5069 - accuracy: 0.8142
Epoch 7/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4959 - accuracy: 0.8195
Epoch 8/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.4915 - accuracy: 0.8195
Epoch 9/10
60000/60000 [================

--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 37us/sample - loss: 1.0977 - accuracy: 0.6157
Epoch 2/10
60000/60000 [==============================] - 2s 32us/sample - loss: 0.7569 - accuracy: 0.7339
Epoch 3/10
60000/60000 [==============================] - 2s 30us/sample - loss: 0.6771 - accuracy: 0.7633
Epoch 4/10
60000/60000 [==============================] - 2s 30us/sample - loss: 0.6365 - accuracy: 0.7764
Epoch 5/10
60000/60000 [==============================] - 2s 30us/sample - loss: 0.6097 - accuracy: 0.7853
Epoch 6/10
60000/60000 [==============================] - 2s 30us/sample - loss: 0.5910 - accuracy: 0.7887
Epoch 7/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5750 - accuracy: 0.7954
Epoch 8/10
60000/60000 [==============================] - 2s 31us/sample - loss: 0.5624 - accuracy: 0.8004
Epoch 9/10
60000/60000 [=================

--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.6018 - accuracy: 0.7904
Epoch 2/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4476 - accuracy: 0.8406
Epoch 3/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.4120 - accuracy: 0.8526
Epoch 4/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3905 - accuracy: 0.8592
Epoch 5/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3790 - accuracy: 0.8619
Epoch 6/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3665 - accuracy: 0.8673
Epoch 7/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.3573 - accuracy: 0.8685
Epoch 8/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.3492 - accuracy: 0.8727
Epoch 9/10
60000/60000 [================

--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.9209 - accuracy: 0.6904
Epoch 2/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.6019 - accuracy: 0.7935
Epoch 3/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5387 - accuracy: 0.8132
Epoch 4/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.5012 - accuracy: 0.8268
Epoch 5/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4766 - accuracy: 0.8325
Epoch 6/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4599 - accuracy: 0.8393
Epoch 7/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4467 - accuracy: 0.8413
Epoch 8/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4363 - accuracy: 0.8453
Epoch 9/10
60000/60000 [=================

--- Starting trial: run-6
{'num_units': 32, 'dropout': 0.2, 'optimizer': 'adam'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 40us/sample - loss: 0.6680 - accuracy: 0.7703
Epoch 2/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4889 - accuracy: 0.8275
Epoch 3/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4556 - accuracy: 0.8365
Epoch 4/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4352 - accuracy: 0.8450
Epoch 5/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4171 - accuracy: 0.8497
Epoch 6/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.4061 - accuracy: 0.8509
Epoch 7/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.3980 - accuracy: 0.8542
Epoch 8/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.3928 - accuracy: 0.8558
Epoch 9/10
60000/60000 [================

--- Starting trial: run-7
{'num_units': 32, 'dropout': 0.2, 'optimizer': 'sgd'}
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.9583 - accuracy: 0.6674
Epoch 2/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.6573 - accuracy: 0.7750
Epoch 3/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.5910 - accuracy: 0.7946
Epoch 4/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.5535 - accuracy: 0.8069
Epoch 5/10
60000/60000 [==============================] - 2s 33us/sample - loss: 0.5289 - accuracy: 0.8162
Epoch 6/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.5118 - accuracy: 0.8193
Epoch 7/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.4965 - accuracy: 0.8262
Epoch 8/10
60000/60000 [==============================] - 2s 36us/sample - loss: 0.4846 - accuracy: 0.8293
Epoch 9/10
60000/60000 [=================

In [23]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6015 (pid 24146), started 0:09:17 ago. (Use '!kill 24146' to kill it.)

In [24]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

In [25]:
%tensorboard --logdir logs/hparam_demo

ERROR: Failed to launch TensorBoard (exited with 255).
Contents of stderr:
E1221 00:42:40.643552 4580650432 program.py:226] TensorBoard could not bind to any port around 6006 (tried 10 times)
ERROR: TensorBoard could not bind to any port around 6006 (tried 10 times)